# Loading the datasets

In [1]:
import os

Get all the data in ./data

In [2]:
files = []
def list_files_recursive(path='.'):
    for entry in os.listdir(path):
        full_path = os.path.join(path, entry)
        if os.path.isdir(full_path):
            list_files_recursive(full_path)
        else:
            files.append(full_path)

# Specify the directory path you want to start from
directory_path = './data'
list_files_recursive(directory_path)

Retrieve the data in two different lists (npz files and csv files)

In [3]:
dataFilesNpz = []
dataFilesCSV = []
for i in files:
    if i[-3:] == "npz" and ("reversed" not in i and "symetric" not in i):
        dataFilesNpz.append(i)
    if i[-3:] == "csv":
        dataFilesCSV.append(i)
    

We gather the usefull information for the npz files (Raycast values, speed and current controls)

In [4]:
import pickle
import lzma
import re
import csv

x = []
y = []

for i in range(len(dataFilesNpz)):
    x.append([])
    y.append([])

for i in dataFilesNpz:
    cli = int(re.search(r"client\d", i).group()[-1]) - 1
    with lzma.open(i, "rb") as file:
        data = pickle.load(file)
        x[cli].append([[e.raycast_distances, e.car_speed] for e in data])
        y[cli].append([e.current_controls for e in data])


        

A little processing on the lists to flatten each sub datasets. Previously we had something like 

[dataset1[dataset1_1, dataset1_2,...], dataset2[dataset2_1, dataset2_2,...],...]

and we want [dataset1[all the data from 1], dataset2[all the data from 2],...]


In [6]:
from functools import reduce
for i in x:
    if len(i) > 0:
        i = reduce(lambda a,b:a+b, i)


for i in y:
    if len(i) > 0:
        i = reduce(lambda a,b:a+b, i)

    

In [7]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

The next function is used to separate features from labels and create the tensors. We also split the data int train and test sets

We can also specify what protion of the data we want to use.

In [8]:
def preprocessData(data_x, data_y, percent_of_data = 100):

    y2 = []
    newList = []
    for idx, j in enumerate(data_x[0]):
        tmp = list(j[0])
        tmp.append(j[1])
        newList.append(tmp)
        newList.append(tmp[::-1])

        y2.append(data_y[0][idx])
        data_y[0][idx] = list(data_y[0][idx])
        tmp2 = data_y[0][idx][2]
        data_y[0][idx][2] = data_y[0][idx][3]
        data_y[0][idx][3] = tmp2

        y2.append(data_y[0][idx])

    tensorX = [torch.Tensor(i) for i in newList]
    tensorY = [torch.Tensor(i) for i in y2]  

    X_train, X_test, y_train, y_test = train_test_split(tensorX, 
                                                    tensorY, 
                                                    test_size=0.2, 
                                                    random_state=42) 
    
    train_select = int(len(X_train) * percent_of_data / 100)
    test_select = int(len(X_test) * percent_of_data / 100)

    X_train = X_train[:train_select]
    y_train = y_train[:train_select]
    X_test = X_test[:test_select]
    y_test = y_test[:test_select]

    return X_train, X_test, y_train, y_test


This class defines the architecture of the neural network

In [9]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      self.layIn = nn.Linear(16, 9)

      self.fc1 = nn.Linear(9, 9)
      self.fc2 = nn.Linear(9, 9)
      self.fc3 = nn.Linear(9, 9)

      self.out = nn.Linear(9,4)
    
    def forward(self, x):
        x = self.layIn(x)
        x = F.sigmoid(x)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.sigmoid(x)

        out = self.out(x)
        return out
    

The next function is called every epoch. We first infer the test data and compare it to the expected result. We then compute the loss function and do the backward propagation to improve the model. After that we have a bit of code to compute the loss and accuracy for this epoch.

In [11]:
def runModel(my_nn, X_train, y_train, X_test, y_test, criterion, optimizer, epoch, verbose = False):
    train_loss = 0
    my_nn.train()
    for idx, i in enumerate(X_train):
      # 1. Forward pass (model outputs raw logits)
      y_logits = my_nn(i)
      y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls

      loss = criterion(y_logits, y_train[idx]) 
      train_loss += loss

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backwards
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

    my_nn.eval()
    test_acc = 0
    test_loss = 0
    train_acc = 0
    
    if epoch%1 == 0: #was used to test each x epoch
      
      for idx, i in enumerate(X_test):
        
        with torch.inference_mode():
          # 1. Forward pass
          test_logits = my_nn(i).squeeze() 
          test_pred = torch.round(torch.sigmoid(test_logits))
          # 2. Caculate loss/accuracy
          test_loss += criterion(test_logits,
                              y_test[idx])
          for idx, j in enumerate(y_test[idx].tolist()):
            if test_pred.tolist()[idx] == j: 
              test_acc+=1

      for idx, i in enumerate(X_train):
        
        with torch.inference_mode():
          # 1. Forward pass
          train_logits = my_nn(i).squeeze() 
          train_pred = torch.round(torch.sigmoid(train_logits))
          # 2. Caculate loss/accuracy

          for idx, j in enumerate(y_train[idx].tolist()):
            if train_pred.tolist()[idx] == j: 
              train_acc+=1
      
      if verbose:
        print(f"epoch {epoch}: Acc: {test_acc/(4*len(X_test))}, loss: {test_loss/len(X_test)}")
      
      return [test_loss/len(X_test), train_loss/len(X_train)]

In [12]:
def getData(pos, percent = 100):
    X_train, X_test, y_train, y_test = preprocessData(x[pos], y[pos], percent)
    return X_train, y_train, X_test, y_test

We now start the federated learning part. We select a small sample from all the data (10% here) and pretrain a base model.

In [13]:
# Pipeline for training
# First base model training
import copy
globalNN = Net()
criterion = nn.CrossEntropyLoss(torch.Tensor([1,1,1,1]))
optimizer = optim.Adam(globalNN.parameters(), lr=0.001)
base_optim = copy.deepcopy(optimizer)

#setup 10% of the data
X_train_start = []
y_train_start = []
X_test_start = []
y_test_start = []

for i in range(6):
    X_train, X_test, y_train, y_test = preprocessData(x[i], y[i], 10)

    X_train_start.append(X_train)
    y_train_start.append(y_train)
    X_test_start.append(X_test)
    y_test_start.append(y_test)


X_train_start = reduce(lambda a,b:a+b, X_train_start)
y_train_start = reduce(lambda a,b:a+b, y_train_start)
X_test_start = reduce(lambda a,b:a+b, X_test_start)
y_test_start = reduce(lambda a,b:a+b, y_test_start)


In [14]:
for epoch in range(51): # Pretrained base model
    runModel(globalNN, X_train_start, y_train_start, X_test_start, y_test_start, criterion, optimizer, epoch, True)

epoch 0: Acc: 0.5219435736677116, loss: 1.586965560913086
epoch 1: Acc: 0.707680250783699, loss: 1.5311529636383057
epoch 2: Acc: 0.731974921630094, loss: 1.3761284351348877
epoch 3: Acc: 0.7523510971786834, loss: 1.32063889503479
epoch 4: Acc: 0.7703761755485894, loss: 1.2734323740005493
epoch 5: Acc: 0.780564263322884, loss: 1.258720874786377
epoch 6: Acc: 0.7594043887147336, loss: 1.2641924619674683
epoch 7: Acc: 0.7735109717868338, loss: 1.2575322389602661
epoch 8: Acc: 0.7664576802507836, loss: 1.244807243347168
epoch 9: Acc: 0.7852664576802508, loss: 1.2388451099395752
epoch 10: Acc: 0.7821316614420063, loss: 1.2474243640899658
epoch 11: Acc: 0.7844827586206896, loss: 1.2253687381744385
epoch 12: Acc: 0.7617554858934169, loss: 1.219712257385254
epoch 13: Acc: 0.7907523510971787, loss: 1.2107442617416382
epoch 14: Acc: 0.7899686520376176, loss: 1.2067759037017822
epoch 15: Acc: 0.7727272727272727, loss: 1.1966116428375244
epoch 16: Acc: 0.7719435736677116, loss: 1.1898484230041504

After we computed the base model, We create workers for each sub datasets. We store them in a list. We have to be careful when lodaing the weights because we will use processes to make them work in parallel

In [15]:
modelsList = []
for i in range(6):
    model_clone = Net()
    model_clone.load_state_dict(copy.deepcopy(globalNN.state_dict()))
    modelsList.append(model_clone)

The next function creates processes and runs the optimisation on each worker. We can see that each process fetches the specific data for its worker.

In [16]:
#creating threads and launching individual data
import concurrent
from concurrent import futures


    
def thread(x):
    try:
        data = getData(x)
        opt = optim.Adam(modelsList[x].parameters(), lr=0.001)
        for i in range(50):
            r = runModel(modelsList[x], data[0], data[1], data[2], data[3], criterion, opt, i)
        return modelsList[x]
    except Exception as e:
        print(f"oh no: {e}")



In [17]:
def runThreads():
    executor = concurrent.futures.ProcessPoolExecutor(10)
    futures = [executor.submit(thread, i) for i in range(6)]
    futuresDone = concurrent.futures.wait(futures)[0]
    print("done")
    return [x.result() for x in futuresDone]

Here we compute the weights average after each round of optimisation.

In [18]:
def getAvgWeights(mod):
    weights = list(map(lambda x: copy.deepcopy(x.state_dict()), mod))
    avg_weights = copy.deepcopy(weights[0])
    for key in avg_weights:
        avg_weights[key] = (weights[0][key] + weights[1][key] + weights[2][key] + weights[3][key] + weights[4][key] + weights[5][key]) / 6
    return avg_weights

This part is meant to launch the worker threads a number of times and create new wokers with the new weights once a round is completed. We finally get the final weigths for the model.

In [19]:
for i in range(15):
    print(f"pass {i} through models")
    modelsList = runThreads()
    print("calc new weights")
    new_weights = getAvgWeights(modelsList)
    modelsList = []
    for i in range(6):
        model_clone = Net()
        model_clone.load_state_dict(copy.deepcopy(new_weights))
        modelsList.append(model_clone)

finalWeights = getAvgWeights(modelsList)
    

pass 0 through models
done
calc new weights
pass 1 through models
done
calc new weights
pass 2 through models
done
calc new weights
pass 3 through models
done
calc new weights
pass 4 through models
done
calc new weights
pass 5 through models
done
calc new weights
pass 6 through models
done
calc new weights
pass 7 through models
done
calc new weights
pass 8 through models
done
calc new weights
pass 9 through models
done
calc new weights
pass 10 through models
done
calc new weights
pass 11 through models
done
calc new weights
pass 12 through models
done
calc new weights
pass 13 through models
done
calc new weights
pass 14 through models
done
calc new weights


In [20]:
print(finalWeights)

OrderedDict({'layIn.weight': tensor([[ 6.7935e-01, -1.7538e-01,  1.1575e+00,  2.6148e-01, -2.6437e-01,
          3.0288e-02,  4.2055e-02, -4.2197e-01, -8.8527e-01, -4.2597e-01,
         -6.2621e-02, -3.5026e-01, -1.4880e-01,  1.0666e-01,  4.0410e-02,
         -3.4778e-01],
        [ 2.8451e-01,  2.5293e-01,  4.0484e-01,  1.4559e-01,  2.8419e-01,
          2.0638e-01,  5.2034e-01,  7.1690e-01,  4.7914e-01,  1.0249e-01,
          2.6836e-01, -2.7233e-01,  3.5176e-01,  1.7120e-01,  5.1377e-01,
          4.7481e-01],
        [ 4.9147e-01,  4.4576e-02, -1.5631e-01, -1.4679e-01, -1.0776e-01,
          9.0382e-02,  3.1270e-01,  2.3470e-01,  6.1215e-01,  4.8834e-01,
          2.7798e-01,  2.9586e-01,  1.6758e-01,  3.2600e-01,  2.7632e-01,
          5.8326e-01],
        [-7.8664e-01, -5.6801e-01, -4.3534e-01, -4.1508e-01, -6.2161e-01,
         -4.0843e-01, -5.4060e-01, -6.0136e-01, -3.3182e-01, -1.2958e-01,
         -1.8029e-01,  4.1025e-01, -1.2986e-01,  5.5606e-02, -2.8897e-01,
         -8.83

In the end, we save the weights to the files to be used in the autopilote

In [25]:
torch.save(finalWeights, "best_FL_model.pt")
finalNN = Net()
finalNN.load_state_dict(finalWeights)

<All keys matched successfully>

In [1]:
torch.sigmoid(finalNN(X_train[15]))
#torch.sigmoid(globalNN(X_train[15]))

NameError: name 'torch' is not defined